## Introduction

Every year, Formula One travels all around the world to host races in different countries. Huge numbers of people and lots of equipment have to be hauled from location to location, causing significant environmental impact.

Formula One needs to do everything it can to lessen its carbon emissions, given that it's a spectacle that entirely revolves around motor cars. Some improvements have been made, such as the move to hybrid engines in 2014, but there's a lot more that could be done. 

One particularly simple change would be to organise the season so that the total distance travelled between race weekends is minimised. As I'll show, the current calendar order is very inefficient and involves back-and-forth trips between continents. I will propose an improved ordering with a lower total distance travelled, and hence improved carbon emissions.

It should be noted that this analysis will be based on straight line distances from racetrack to racetrack, and assumes that the same number of people/amount of equipment is being hauled each time. In reality, I imagine that there's a lot of variation in what is hauled, where it's hauled via, and how it's hauled, so in that regard this will be a fairly simplistic exercise. I nonetheless expect it to be indicative of the kind of improvements F1 could make.

### Loading and exploring the data

I'll be using the handy [Ergast API](http://ergast.com/mrd/) to get a season's schedule with longitudes and latitudes.

I'll then calculate the distances using GeoPandas, and visualise these journeys using Plotly's mapping functionality.

In [10]:
import requests
import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go

pd.options.mode.chained_assignment = None

# Function to return a geodataframe with distances for a given season
def get_season_geodataframe(season):

    # Call Ergast API and insert results into a dataframe
    api_url = "https://ergast.com/api/f1/" + str(year) + ".json"

    raw_result = requests.get(api_url).json()
    race_list = raw_result["MRData"]["RaceTable"]["Races"]
    races_df = pd.DataFrame(race_list)

    # Extract longitude and latitude from circuit dict
    for coord in ["long", "lat"]:
        races_df[coord] = races_df["Circuit"].apply(lambda x: x["Location"][coord])

    # Ignore irrelevant fields
    races_df = races_df[["season", "round", "raceName", "date", "long", "lat"]]

    # Add point geometry data based on longs and lats
    races_gdf = gpd.GeoDataFrame(races_df, geometry=gpd.points_from_xy(races_df["long"], races_df["lat"], crs="EPSG:4326"))
    
    # Convert projection to UTM to allow distance calculation in metres
    races_gdf.to_crs(epsg=3310, inplace=True)

    # Calculate distances to each race from its preceding event
    races_gdf["distance_in_km"] = 0
    for i in range(1, len(races_gdf.index)):
        races_gdf["distance_in_km"].iloc[i] = races_gdf["geometry"].iloc[i].distance(races_gdf["geometry"].iloc[i-1]) / 1000

    return races_gdf


# Example of function output based on this season
get_season_geodataframe(2022).head(10)

,season,round,raceName,date,long,lat,geometry,distance_in_km
0,2022,1,Bahrain Grand Prix,2022-03-20,50.5106,26.0325,POINT (9387755.676 10487787.397),0.000000
1,2022,2,Saudi Arabian Grand Prix,2022-03-27,39.1044,21.6319,POINT (10054849.388 9396447.022),1279.076946
2,2022,3,Australian Grand Prix,2022-04-10,144.968,-37.8497,POINT (-12885849.420 81620.359),24759.678059
3,2022,4,Emilia Romagna Grand Prix,2022-04-24,11.7167,44.3439,POINT (7485923.654 6938848.810),21494.899865
4,2022,5,Miami Grand Prix,2022-05-08,-80.2389,25.9581,POINT (3927235.440 -496062.467),8242.703895
5,2022,6,Spanish Grand Prix,2022-05-22,2.26111,41.57,POINT (7606875.711 6115107.470),7566.195904
6,2022,7,Monaco Grand Prix,2022-05-29,7.42056,43.7347,POINT (7481570.670 6586027.465),487.305854
7,2022,8,Azerbaijan Grand Prix,2022-06-12,49.8533,40.3725,POINT (7855510.208 10075480.852),3509.432393
8,2022,9,Canadian Grand Prix,2022-06-19,-73.5228,45.5,POINT (3523403.422 1710521.775),9420.174604
9,2022,10,British Grand Prix,2022-07-03,-1.01694,52.0786,POINT (6433981.588 6210396.875),5359.136234


In [17]:
# Plot this season's route around the globe
season = 2022

gdf = get_season_geodataframe(season)

total_distance = int(gdf["distance_in_km"].sum())

fig = go.Figure()

fig.add_trace(
    go.Scattergeo(
        lon=gdf["long"],
        lat=gdf["lat"],
        mode="lines+markers",
        text=gdf["raceName"]
    )
)

fig.update_layout(
    geo_projection_type="orthographic",
    title="F1 " + str(season) + " calendar, total distance: " + str(total_distance) + "km"
)


We can clearly see from the above plot that there are some unnecessarily long trips between events, e.g. repeated trips across the Atlantic to get to Miami and Montreal in between European races.